<a href="https://colab.research.google.com/github/Nishasathish13/TheSchoolofAI-END3.0/blob/main/Session%204%20-%20RNN%20are%20dead%2C%20and%20HandsOn/Assignment-part1/END2_Session_4_RNN_to_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#Using spacy Tokenizer
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext.legacy import datasets

#Downloadingg the IMDB dataset
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 29.8MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['Two', 'dysfunctional', 'brothers', '(', 'Philip', 'Seymour', 'Hoffman', 'and', 'Ethan', 'Hawke', ')', 'get', 'tired', 'of', 'competing', 'for', 'who', 'is', 'the', 'bigger', 'f***-up', 'and', 'who', 'Daddy', '(', 'Albert', 'Finney', ')', 'loves', 'more', ',', 'so', 'they', 'hatch', 'a', 'hair', '-', 'brained', 'scheme', 'to', 'rob', 'Mommy', 'and', 'Daddy', "'s", 'jewelry', 'store', 'so', 'that', 'they', 'can', 'clear', 'their', 'debts', 'and', 'start', 'fresh', '.', 'Sounds', 'like', 'a', 'great', 'plan', 'except', 'that', 'this', 'is', 'a', 'suspenseful', '1970', "'s", 'style', 'melodrama', 'about', 'a', 'heist', 'gone', 'wrong', ',', 'and', 'boy', ',', 'do', 'things', 'really', 'go', 'wrong', 'here', 'for', 'our', 'hapless', 'duo', 'and', 'everyone', 'involved', '.', 'Lasciviously', 'concocted', 'by', 'screenwriter', 'Kelly', 'Masterson', 'and', 'classically', 'executed', 'by', 'director', 'Sidney', 'Lumet', ',', '"', 'Before', 'the', 'Devil', 'Knows', 'You', "'re", 'Dead

In [5]:
import random

#splitting into train and validation data
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [7]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
#Top20 nmost frequent words
print(TEXT.vocab.freqs.most_common(20))

[('the', 202755), (',', 192304), ('.', 166205), ('and', 109267), ('a', 108951), ('of', 100755), ('to', 93415), ('is', 76409), ('in', 61231), ('I', 54331), ('it', 53702), ('that', 49281), ('"', 44129), ("'s", 43421), ('this', 42424), ('-', 37156), ('/><br', 35492), ('was', 35161), ('as', 30491), ('with', 29823)]


In [10]:
#Returns List mapping indices to tokens.
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [11]:
#Returns dictionary mapping tokens to indices
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [12]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

##RNN CODE

In [13]:
# import torch.nn as nn

# class RNN(nn.Module):
#     def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
#         super().__init__()
        
#         self.embedding = nn.Embedding(input_dim, embedding_dim)
        
#         self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
#         self.fc = nn.Linear(hidden_dim, output_dim)
        
#     def forward(self, text):

#         #text = [sent len, batch size]
        
#         embedded = self.embedding(text)
        
#         #embedded = [sent len, batch size, emb dim]
        
#         output, hidden = self.rnn(embedded)
        
#         #output = [sent len, batch size, hid dim]
#         #hidden = [1, batch size, hid dim]
        
#         assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
#         return self.fc(hidden.squeeze(0))

##LSTM CODE

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        #Embedding layer
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        #LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                            hidden_dim,
                            batch_first=False)
        
        #Fullyconnected/Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        
        # text = [batch size, sent_length]
        #print("text :", text.shape)
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
        #print("embedded :", embedded.shape)
        # packed sequence, padding 
        # packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        #three outputs for LSTMs
        packed_output, (hidden, cell) = self.encoder(embedded)
        # Hidden = [batch size, hid dim * num directions]
        #print("hidden :", hidden.shape)
        dense_outputs = self.fc(hidden)   
        #print("dense_outputs :", dense_outputs.shape)
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
        #print("output :", output.shape)    
        return output
        # return self.fc(hidden).squeeze(0)  

In [15]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

#28.5 million trainable parameters

The model has 2,867,049 trainable parameters


In [17]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [18]:
criterion = nn.BCEWithLogitsLoss()

In [19]:
model = model.to(device)
criterion = criterion.to(device)

In [20]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [21]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        # print(batch.text.squeeze().shape)
        # print(batch.text.shape)
        # resets the gradients after every batch
        optimizer.zero_grad()

        predictions = model(batch.text).squeeze()
        #print(predictions)
        #print(predictions.shape, batch.label.shape)
        loss = criterion(predictions, batch.label)
    
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [24]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 19m 35s
	Train Loss: 0.817 | Train Acc: 49.62%
	 Val. Loss: 0.806 |  Val. Acc: 50.74%
Epoch: 02 | Epoch Time: 21m 51s
	Train Loss: 0.816 | Train Acc: 49.68%
	 Val. Loss: 0.806 |  Val. Acc: 50.74%
Epoch: 03 | Epoch Time: 22m 26s
	Train Loss: 0.816 | Train Acc: 49.68%
	 Val. Loss: 0.806 |  Val. Acc: 50.74%
Epoch: 04 | Epoch Time: 20m 0s
	Train Loss: 0.817 | Train Acc: 49.65%
	 Val. Loss: 0.806 |  Val. Acc: 50.74%
Epoch: 05 | Epoch Time: 19m 34s
	Train Loss: 0.817 | Train Acc: 49.67%
	 Val. Loss: 0.806 |  Val. Acc: 50.74%


In [25]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.813 | Test Acc: 50.01%


In [ ]:
#Results from RNN Model

# Epoch: 01 | Epoch Time: 10m 45s
# 	Train Loss: 0.694 | Train Acc: 49.16%
# 	 Val. Loss: 0.696 |  Val. Acc: 50.36%
# Epoch: 02 | Epoch Time: 10m 40s
# 	Train Loss: 0.693 | Train Acc: 50.31%
# 	 Val. Loss: 0.695 |  Val. Acc: 49.04%
# Epoch: 03 | Epoch Time: 10m 39s
# 	Train Loss: 0.693 | Train Acc: 50.50%
# 	 Val. Loss: 0.696 |  Val. Acc: 50.02%
# Epoch: 04 | Epoch Time: 10m 43s
# 	Train Loss: 0.693 | Train Acc: 50.63%
# 	 Val. Loss: 0.696 |  Val. Acc: 50.15%
# Epoch: 05 | Epoch Time: 10m 38s
# 	Train Loss: 0.693 | Train Acc: 50.37%
# 	 Val. Loss: 0.695 |  Val. Acc: 49.17%

# Test Loss: 0.706 | Test Acc: 47.98%